## Import libraries

In [50]:
import ee
import geemap
import os, re, json, time
import geopandas as gpd
from datetime import datetime, timedelta, date
from shapely.geometry import Polygon, mapping
from timezonefinder import TimezoneFinder
import pytz
from zoneinfo import ZoneInfo


## Start Connection with Google Earth Engine

In [ ]:
ee.Authenticate(force=True)
ee.Initialize()
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


## Set up environmental variables

In [3]:
# To get root path under project folder structure
root_folder = os.getcwd().split('\\')
root_folder = root_folder[1:-3]
if root_folder[-1] != 'fireRunSeverity':
    print("Didn't get correct root folder!")
    print(root_folder)
root_folder = os.path.join('C:\\', *root_folder)

# List in put data for GEE fetch
run_DataDir = r"data/fireruns"
AreaList = os.listdir(os.path.join(root_folder,run_DataDir))
AreaList
in_Name = 'GIF14_Au'

# os.path.basename(root_folder)

## Load EE side data

In [26]:
# Add Earth Engine dataset
S2_harmon = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
Map = geemap.Map(center=[40, -100], zoom=4)
Map.setCenter(148.3462, -37.3684,9)


## Load Local side data

In [11]:
dataFLD = os.path.join(root_folder, 'data', 'fireruns', in_Name)
inFLD = os.path.join(dataFLD, 'input')
shpIn  = []
for f in os.scandir(inFLD):
    if re.search(r".shp$", f.name):
        print(f.name)
        shpIn.append(f)

if len(shpIn) != 1:
    print("Check No. of shp file not equal to 1!")
else:
    shpIn = shpIn[0]

# Read Shp file
shpGPD = gpd.read_file(os.path.join(inFLD, shpIn))
print(shpGPD.crs)
shp_reproj = shpGPD.to_crs("EPSG:4326")
# Get the bounds
minx, miny, maxx, maxy = shp_reproj.total_bounds
# Create a polygon of bounds
bounds_polygon = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny), (minx, miny)])
# Convert the polygon to GeoJSON
json_polygon = mapping(bounds_polygon)
print(json_polygon)

GIF14_32755_multipart.shp
EPSG:32755
{'type': 'Polygon', 'coordinates': (((148.797548490504, -37.78877807241547), (148.797548490504, -37.04608874283253), (150.0295287036503, -37.04608874283253), (150.0295287036503, -37.78877807241547), (148.797548490504, -37.78877807241547)),)}


## Modify time zone

In [12]:
def get_timezone(lat, lon):
    tf = TimezoneFinder()
    tz_name = tf.timezone_at(lng=lon, lat=lat)
    return tz_name

latitude = (miny + maxy)/2
longitude = (minx + maxx)/2
timezone = get_timezone(latitude, longitude)
print(f"Accurate Time Zone: {timezone}")

Accurate Time Zone: Australia/Melbourne


In [28]:
# Get date of data
# Convert time zone to UTC
fire_dtString = list(shpGPD['FeHo'])
fire_dtsList  = [datetime.strptime(dt, "%Y/%m/%d_%H%M").replace(tzinfo=ZoneInfo(timezone)) for dt in fire_dtString]
tz = pytz.timezone("UTC")
fire_dtsList_UTC = [dt.astimezone(tz) for dt in fire_dtsList]

print(sorted(fire_dtsList_UTC))
dateSt = min(fire_dtsList_UTC).date()
dateEd = max(fire_dtsList_UTC).date()
print("Fire start date and end date:")
print(dateSt, dateEd)


# Compute the date period for retrieving Satellite img
prefire_date  = [dateSt - timedelta(days = 4*30), dateSt - timedelta(days = 1)]
postfire_date = [dateEd + timedelta(days = 1),    dateEd + timedelta(days = 4*30)]

prefire_date = [d.strftime("%Y-%m-%d") for d in prefire_date]
postfire_date = [d.strftime("%Y-%m-%d") for d in postfire_date]

print("Pre- and Post- fire periods selected for satellite images:")
print(prefire_date, postfire_date)

[datetime.datetime(2019, 12, 29, 14, 0, tzinfo=<UTC>), datetime.datetime(2019, 12, 30, 3, 0, tzinfo=<UTC>), datetime.datetime(2019, 12, 31, 14, 0, tzinfo=<UTC>), datetime.datetime(2019, 12, 31, 14, 0, tzinfo=<UTC>), datetime.datetime(2019, 12, 31, 14, 0, tzinfo=<UTC>), datetime.datetime(2019, 12, 31, 14, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 14, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 14, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 14, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 2, 3, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 2, 3, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 2, 3, 0, tzinfo=<UTC>), datetime.d

## Upload data to EE

In [27]:
clipAOI = ee.Geometry.Polygon(json_polygon["coordinates"])
clipAOI2 = clipAOI.buffer(50000)
print(clipAOI.getInfo())

{'type': 'Polygon', 'coordinates': [[[148.797548490504, -37.78877807241547], [150.0295287036503, -37.78877807241547], [150.0295287036503, -37.04608874283253], [148.797548490504, -37.04608874283253], [148.797548490504, -37.78877807241547]]]}


In [10]:
Map.addLayer(clipAOI2, {'color': "blue"}, "Input SHP range")

## (GEE) Process satellite

In [27]:
# filter satellite data
bandList = ["B.", "B..", "QA60", "MSK_CLDPRB"]
S2_pre_select = S2_harmon.filterDate(prefire_date[0], prefire_date[1]) \
.filterBounds(clipAOI2).select(bandList)
# .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))

S2_pos_select = S2_harmon.filterDate(postfire_date[0], postfire_date[1]) \
.filterBounds(clipAOI2).select(bandList)
# .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))

In [ ]:
# Functions for Computing Necessary indices
def func_maskClouds(image):
    imgCLP = image.select("MSK_CLDPRB")
    mskCLP = imgCLP.lt(10);  
    qa = image.select("QA60")
    Clouds = qa.bitwiseAnd(1 << 10).eq(0);  
    Cirrus = qa.bitwiseAnd(1 << 11).eq(0); 
    mask = Clouds.And(Cirrus).And(mskCLP)
    return image.updateMask(mask)

def func_rescale(image):
    refl = image.multiply(0.0001)
    return refl.copyProperties(image, ["system:time_start"])

def func_calcIndices (image):
    NBR  = image.normalizedDifference(["B8", "B12"]).rename("NBR")
    NDVI = image.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return image.addBands([NBR, NDVI])

def func_calcIndices2(image):
    RBR   = image.expression(
        "dNBR / (preNBR + 1.001)", {
            "preNBR": image.select('preNBR'),
            "dNBR": image.select('Cl_dNBR') 
        }
    ).rename('Cl_RBR').toFloat()
    RdNBR = image.expression(
        "(abs(preNBR) < 0.001) ? dNBR/sqrt(0.001) : dNBR/sqrt(abs(preNBR))",{
            "preNBR": image.select('preNBR'),
            "dNBR": image.select('Cl_dNBR')
        }
    ).rename("Cl_RdNBR").toFloat()
    M_RBR   = image.expression(
        "dNBR / (preNBR + 1.001)", {
            "preNBR": image.select('M_preNBR'),
            "dNBR": image.select('M_dNBR') 
        }
    ).rename('M_RBR').toFloat()
    M_RdNBR = image.expression(
        "(abs(preNBR) < 0.001) ? dNBR/sqrt(0.001) : dNBR/sqrt(abs(preNBR))",{
            "preNBR": image.select('M_preNBR'),
            "dNBR": image.select('M_dNBR')
        }
    ).rename("M_RdNBR").toFloat()
    return image.addBands([RBR, RdNBR, M_RBR, M_RdNBR])

In [34]:
S2pre = S2_pre_select.map(func_maskClouds).map(func_rescale) \
.sort('system:time_start') \
.mosaic(); 

S2pos = S2_pos_select.map(func_maskClouds).map(func_rescale) \
.sort('system:time_start', False) \
.mosaic(); 
# print(json.dumps(S2pre.getInfo(), indent=4))

S2pre_mean = S2_pre_select.map(func_maskClouds).map(func_rescale) \
.sort('system:time_start') \
.mean(); 

S2pos_mean = S2_pos_select.map(func_maskClouds).map(func_rescale) \
.sort('system:time_start', False) \
.mean(); 
# print(json.dumps(S2pre.getInfo(), indent=4))


In [14]:
# Visualization
visP = {'bands': ["B4",  "B3",  "B2"], 'max': 0.3}
Map.remove(clipAOI2)
Map.addLayer(S2pre.clip(clipAOI), visP, "S2 pre fire")
Map.addLayer(S2pos.clip(clipAOI), visP, "S2 after fire")

In [ ]:
# Indices Calculation
S2pre_id = func_calcIndices(S2pre)
S2pos_id = func_calcIndices(S2pos)
S2pre_id_m = func_calcIndices(S2pre_mean)
S2pos_id_m = func_calcIndices(S2pos_mean)

# dNBR
S2_idcs1  = S2pre_id.select("NBR").subtract(S2pos_id.select("NBR")).multiply(1000).rename("Cl_dNBR")
S2_idcs1  = S2_idcs1.addBands(S2pre_id_m.select("NBR").subtract(S2pos_id_m.select("NBR")).multiply(1000).rename("M_dNBR"))

# Other indices
S2_idcs1  = S2_idcs1.addBands([S2pre_id.select("NBR").rename("preNBR"), 
                             S2pos_id.select("NBR").rename("posNBR"),
                             S2pre_id_m.select("NBR").rename("M_preNBR"), 
                             S2pos_id_m.select("NBR").rename("M_posNBR"),])

S2_idcs2 = func_calcIndices2(S2_idcs1)
S2_idcs2


## Output image to Google drive folder

In [65]:
# Select needed bands
bd = S2_idcs2.bandNames()
bd = bd.removeAll(['preNBR', 'posNBR', 'M_preNBR', 'M_posNBR'])
S2_idcs = S2_idcs2.select(bd)
bd = S2_idcs.bandNames().getInfo()
bd

['dNBR', 'M_dNBR', 'RBR', 'RdNBR', 'M_RBR', 'M_RdNBR']

In [66]:
# Define export parameters
print("#-----------------#")
print("Exporting for", in_Name)
driveFLD = os.path.join('EarthEngineFolder', in_Name, in_Name)
tStamp = datetime.now().strftime("%d%m%Y_%H%M")
for bd_i in bd:
    export_task = ee.batch.Export.image.toDrive(
        image=S2_idcs.select(bd_i),
        description='sentinel2_fire_indices_image_export',
        folder='EarthEngineFolder',  # Folder in your Google Drive
        fileNamePrefix='PythonGEE_output--'+in_Name+'--S2_'+bd_i+'--'+tStamp,  # File name prefix
        region=clipAOI,  # Define the region to export (could be an area of interest)
        scale=10,  # Spatial resolution in meters
        fileFormat='GeoTIFF',  # File format
        maxPixels=1e12  # Max pixels to export
    )

    # Start the export task
    try:
        export_task.start()
        print("Band: ", bd_i)
        print("Exporting to Google Drive...")

        # Check the status of the export task
        print("Exporting... Please wait")
        while export_task.active():
            print('.',  end="")
            time.sleep(10)

        print('Export completed.')

    except ee.ee_exception.EEException as e:
        print(f"Error during export: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    print("----")


#-----------------#
Exporting for GIF14_Au
Band:  dNBR
Exporting to Google Drive...
Exporting... Please wait
...................................................................Export completed.
----
Band:  M_dNBR
Exporting to Google Drive...
Exporting... Please wait
........................................................................Export completed.
----
Band:  RBR
Exporting to Google Drive...
Exporting... Please wait
......................................Export completed.
----
Band:  RdNBR
Exporting to Google Drive...
Exporting... Please wait
.......................................Export completed.
----
Band:  M_RBR
Exporting to Google Drive...
Exporting... Please wait
..................................................Export completed.
----
Band:  M_RdNBR
Exporting to Google Drive...
Exporting... Please wait
...................................................Export completed.
----


## Miscellaneous

#### Miscellaneous: Visualize calculated indices

In [ ]:
# Produce Severity classes
S2_dNBR = S2_idcs.select("dNBR")
S2_severity = ee.Image(1) \
.where(S2_dNBR.gt(-500).And(S2_dNBR.lte(-251)), 1) \
.where(S2_dNBR.gt(-251).And(S2_dNBR.lte(-101)), 2) \
.where(S2_dNBR.gt(-101).And(S2_dNBR.lte(99)), 3) \
.where(S2_dNBR.gt(99).And(S2_dNBR.lte(269)), 4) \
.where(S2_dNBR.gt(269).And(S2_dNBR.lte(439)), 5) \
.where(S2_dNBR.gt(439).And(S2_dNBR.lte(659)), 6) \
.where(S2_dNBR.gt(660).And(S2_dNBR.lte(1300)), 7) \
.clip(clipAOI)
# # ------------------------------------------------ # #



# # ------------------------------------------------ # #
# # NBR fire severity color:https:#un-spider.Org/advisory-support/recommended-practices/recommended-practice-burn-severity/Step-By-Step/RStudio
dnbrPal = ["556B2F","6E8B3D","32CD32", "EEEE00", "EE7600", "FF0000", "A020F0"]
# # NDVI palette https:#custom-scripts.sentinel-hub.com/sentinel-2/ndvi/
ndviPal = ["0c0c0c", "eaeaea", "ccc682", "91bf51", "70a33f", "4f892d", "306d1c", "0f540a", "004400"]
# # name of the legend
dnbrNames = ["Enhanced Regrowth, High", "Enhanced Regrowth, Low", "Unburned", "Low Severity", "Moderate-low Severity", "Moderate-high Severity", "High Severity"]

visNBR = {'bands': "NBR", 'max':1, 'min': -1, 'palette': ["green", "red"]}

visNDVI = {
    'bands': "NDVI",
    'max': 1,
    'min': -1,
    'palette': ndviPal
}
visdNBR = {
    'max': 1300,
    'min': -500,
    'palette': dnbrPal
}
visSevere = {
    'max': 7,
    'min': 1,
    'palette': dnbrPal
}

# m.addLayer(S2pre_id, visNDVI, "NDVI_pre")
# m.addLayer(S2pos_id, visNDVI, "NDVI_pos")
Map.addLayer(S2_dNBR.clip(clipAOI), visdNBR, "Cl_dNBR")
Map.addLayer(S2_severity.clip(clipAOI), visSevere, "dNBR Severity")
# # ------------------------------------------------ # #

Map




Map(center=[-37.3684, 148.3462], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

#### Miscellaneous: Counting pixels

In [ ]:
# pixel_count = S2_dNBR.select('dNBR').reduceRegion(
#     reducer=ee.Reducer.count(),
#     geometry=clipAOI,  # Use the image's geometry as the region
#     scale=10,  # Spatial resolution in meters
#     maxPixels=1e10  # Maximum number of pixels to process
# )

# # Print the pixel count
# print('Pixel count for B4 band:', pixel_count.getInfo())

Pixel count for B4 band: {'dNBR': 113379278}
